# Batch Normalization

## Introdução

Batch Normalization é uma técnica que permite normalizar automaticamente os valores que atravessam uma camada da rede neural. Normalmente o uso do BatchNorm permite um treinamento mais rápido e com menores cuidados na inicialização dos pesos. Permite que redes profundas sejam treinadas mais facilmente. 

A normalização é feita para que o resultado tenha média zero e variância unitária, porém, em seguida, o resultado é
escalado pelo fator $\gamma$ e somado ao fator $\beta$. Estes dois fatores são parâmetros que serão também otimizados durante o treinamento do gradiente descendente.

Atualmente a técnica de batch normalization é utilizada em todas as redes profundas. 

A camada de batch normalization é colocada entre a camada densa ou convolucional e antes da camada de ativação.

**Referência:**
- Ioffe, S. and Szegedy, C. (2015), Batch Normalization: Accelerating Deep Network Training by Reducing Internal Covariate Shift., in Francis R. Bach & David M. Blei, ed., 'ICML' , JMLR.org, , pp. 448-456. 
    [PDF:arxiv](http://arxiv.org/pdf/1502.03167.pdf) 

<img src='../figures/batchnorm_neuronios.png', width=900></img>

<img src='../figures/batchnorm_equations.png',width=500></a>

## Diferença entre fase de treinamento e fase de predição ou avaliação

A normalização ocorre de forma distinta na fase de treinamento e na fase de avaliação:
- em treinamento: a média e variância ($\mu$ e $\sigma^2$) são estimados a partir dos valores
das amostras no mini-batch:
    - `running_mean`
    - `running_var`
- em avaliação: a média e variância é calculada pela média móvel, definida pelo parâmetro momentum ($\lambda$):

\begin{align*} 
\boldsymbol{m}_{t} &=  \lambda \boldsymbol{\mu_t} + (1 - \lambda) \boldsymbol{m}_{t-1} &&
\boldsymbol{v}_{t} =   \lambda \boldsymbol{\sigma^{2}_t} + (1 - \lambda) \boldsymbol{v}_{t-1} 
\end{align*}

## Importações

In [1]:
%matplotlib inline
import matplotlib.pyplot as plot
from IPython import display

import sys, copy
import numpy as np
import numpy.random as nr
import pandas as pd

import torch
from torch.autograd import Variable

np.set_printoptions(precision=2, suppress=True)
nr.seed(23456)


## nn.BatchNorm1d - Entrada com dimensão 2

- Documentação oficial: http://pytorch.org/docs/master/nn.html#normalization-layers

Quando a entrada da camada tem duas dimensões (amostras e atributos) a normalização por atributo. Ou seja, calcula-se a estatística (média e variância) para cada coluna da matriz de dados, em cada *mini-batch*.

## Criação rede com apenas uma camada BatchNorm

In [2]:
torch.manual_seed(1234)

momentum = 0.1
model = torch.nn.BatchNorm1d(5, momentum=momentum)        # Rede formada de apenas uma camada BatchNorm

## Parâmetros iniciais, mv_mean = 0. e mv_var = 1.

In [3]:
print('Training:', model.training)
gamma, beta, mv_mean, mv_var = copy.deepcopy(model.state_dict()).values()
param_dict = model.state_dict()
for name,value in param_dict.items():
    print(name)
    print(value.numpy())

Training: True
weight
[ 0.19  0.5   0.62  0.82  0.44]
bias
[ 0.  0.  0.  0.  0.]
running_mean
[ 0.  0.  0.  0.  0.]
running_var
[ 1.  1.  1.  1.  1.]


## Entrada x (não normalizada)

In [4]:
x = Variable(torch.randn(4, 5)) * 100. # Entrada x
print('x:\n',x)

x:
 Variable containing:
-133.6756 -113.6071   23.2839 -172.4477   51.1140
 -61.2595   47.0261   65.2737   57.1934  170.6041
 100.2897 -231.1064  110.7505  171.6769   70.1500
  62.7425  117.5225    9.5521 -151.1595   12.5337
[torch.FloatTensor of size 4x5]



## Forward em treinamento: faz predict usando média e var do mini-batch e atualiza mv_mean e mv_var

In [5]:
print('Training:', model.training)
y = model(x)                           # Forward

Training: True


## Parâmetros: running_mean e running_var estimados com momentum 10% atual e 90% do anterior

In [6]:
print('Training:', model.training)
param_dict = model.state_dict()
for name,value in param_dict.items():
    print(name)
    print(value.numpy())

Training: True
weight
[ 0.19  0.5   0.62  0.82  0.44]
bias
[ 0.  0.  0.  0.  0.]
running_mean
[-0.8  -4.5   5.22 -2.37  7.61]
running_var
[ 1179.64  2475.06   209.37  2770.49   455.28]


## Conferindo os novos valores de running_mean e running_var

\begin{align*} 
\boldsymbol{m}_{t} &=  \lambda \boldsymbol{\mu_t} + (1 - \lambda) \boldsymbol{m}_{t-1} &&
\boldsymbol{v}_{t} =   \lambda \boldsymbol{\sigma^{2}_t} + (1 - \lambda) \boldsymbol{v}_{t-1} 
\end{align*}

In [7]:
print('old mv_mean, mv_var:', mv_mean.numpy(), mv_var.numpy())

mv_mean_new = momentum * x.data.mean(0) + (1 - momentum) * mv_mean
mv_var_new  = momentum * x.data.var(0,unbiased=True) + (1 - momentum) * mv_var

print('my new running mean:',mv_mean_new.numpy())
print('my new running var :',mv_var_new.numpy())

old mv_mean, mv_var: [ 0.  0.  0.  0.  0.] [ 1.  1.  1.  1.  1.]
my new running mean: [-0.8  -4.5   5.22 -2.37  7.61]
my new running var : [ 1179.64  2475.06   209.37  2770.49   455.28]


## Saída  y

In [8]:
y

Variable containing:
-0.2560 -0.2505 -0.4552 -0.8442 -0.1874
-0.1085  0.3364  0.2055  0.4589  0.7086
 0.2205 -0.6798  0.9209  1.1086 -0.0446
 0.1440  0.5939 -0.6712 -0.7234 -0.4766
[torch.FloatTensor of size 4x5]

## Conferindo com código próprio

Em treinamento, a normalização é feita com a estatística do mini-batch

$$  y = \frac{x - mean[x]}{ \sqrt{Var[x] + \epsilon}} * gamma + beta $$

In [9]:
mean = x.mean(dim=0).data
var  = x.var(dim=0, unbiased=False).data
print('mean:',mean.numpy())
print('var: ',var.numpy())
y2 = (x.data - mean) / torch.sqrt(var + model.eps) * gamma + beta

print(y2)

mean: [ -7.98 -45.04  52.22 -23.68  76.1 ]
var:  [  8840.52  18556.22   1563.52  20771.92   3407.85]

-0.2560 -0.2505 -0.4552 -0.8442 -0.1874
-0.1085  0.3364  0.2055  0.4589  0.7086
 0.2205 -0.6798  0.9209  1.1086 -0.0446
 0.1440  0.5939 -0.6712 -0.7234 -0.4766
[torch.FloatTensor of size 4x5]



## Modo predict sem treinamento

In [10]:
model.training = False
y = model(x)                           # Forward

Observe que agora, o running_mean e running_var serão utilizados

In [11]:
print('Training:', model.training)
param_dict = model.state_dict()
for name,value in param_dict.items():
    print(name)
    print(value.numpy())

Training: False
weight
[ 0.19  0.5   0.62  0.82  0.44]
bias
[ 0.  0.  0.  0.  0.]
running_mean
[-0.8  -4.5   5.22 -2.37  7.61]
running_var
[ 1179.64  2475.06   209.37  2770.49   455.28]


## Resultado da rede no modo eval(), note que os valores não ficaram tão normalizados (por quê?)

In [12]:
y

Variable containing:
-0.7410 -1.0914  0.7766 -2.6427  0.8925
-0.3371  0.5155  2.5819  0.9255  3.3438
 0.5637 -2.2668  4.5371  2.7043  1.2830
 0.3543  1.2207  0.1862 -2.3119  0.1010
[torch.FloatTensor of size 4x5]

## Conferindo com código próprio, note que usa-se o running_mean e running_var

In [13]:
gamma, beta, mv_mean, mv_var = model.state_dict().values()

mean = mv_mean
var  = mv_var

y2 = (x.data - mean) / torch.sqrt(var + model.eps) * gamma + beta

print(y2)


-0.7410 -1.0914  0.7766 -2.6427  0.8925
-0.3371  0.5155  2.5819  0.9255  3.3438
 0.5637 -2.2668  4.5371  2.7043  1.2830
 0.3543  1.2207  0.1862 -2.3119  0.1010
[torch.FloatTensor of size 4x5]



## Exercícios

1. Coloque a rede no modo treinamento em um laço, de modo que a running_mean e running_var se aproximem da média e variância do mini-batch. Quantas execuções da rede serão necessárias?

In [14]:
model.training = True
#for i in range(100):
    #y = model(x)
    # Busque aqui os valores da média móvel da média e variância que estão sendo aprendidos
    #
    #
    #print(mv_mean.numpy(), mv_var.numpy())